In [60]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [61]:
import pandas as pd
train_data = pd.read_csv('../Dataset/cleaned_train.csv', encoding='utf-8')
test_data = pd.read_csv('../Dataset/cleaned_dev.csv', encoding='utf-8')

In [62]:
# from preprocessing import clean_data
# train_data = clean_data(train_data)
# test_data = clean_data(test_data)
# # Save the data to a csv file
# train_data.to_csv('../Dataset/cleaned_train.csv', index=False)
# test_data.to_csv('../Dataset/cleaned_dev.csv', index=False)

In [63]:
print(train_data['text'].head(5))

0    بيل غيتس يتلقى لقاح تصوير الابرة السيرنجة الدو...
1    وزير الصحة لحد اليوم وتحديدا هلأ بمؤتمروا الصح...
2    قولكن رح يكونو اد المسؤولية لبنان يوصل اللقاح ...
3    وزير الصحة فخر الدين قوجة يتلقى جرعة لقاح كورو...
4    وئام وهاب يشتم الدول الخليجية طلة اعلامية ويتس...
Name: text, dtype: object


In [64]:
from sklearn.model_selection import train_test_split
# Unpack the data into text, category and stance
Train_X, Test_X, cat_Train_Y, cat_Test_Y, stance_Train_Y, stance_Test_Y = train_data['text'], test_data['text'], train_data['category'], test_data['category'], train_data['stance'], test_data['stance']


In [65]:
bert_encode_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'
bert_preprocess_url='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [66]:
bert_proprocess=hub.KerasLayer(bert_preprocess_url)
bert_encoder=hub.KerasLayer(bert_encode_url)

In [67]:
from sklearn.metrics.pairwise import cosine_similarity
# get the sentence embedding for the sentences
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
# preprocessing the data using bert preprocess layer
preprocessed_text=bert_proprocess(text_input)

# get the bert encoder layer
outputs = bert_encoder(preprocessed_text)

# Nueral network layer
l=tf.keras.layers.Dropout(0.1,name='dropout')(outputs['pooled_output'])
l=tf.keras.layers.Dense(1, activation='sigmoid', name='output')(l)

# construct final model
model = tf.keras.Model(inputs=[text_input], outputs=[l])

In [68]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_4 (KerasLayer)     {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

In [70]:
METRICS=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),tf.keras.metrics.Precision(name='precision'),tf.keras.metrics.Recall(name='recall')]

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=METRICS)
history = model.fit(Train_X,stance_Train_Y,epochs=15)


Epoch 1/15
219/219 [==============================] - 56s 244ms/step - loss: 0.5970 - accuracy: 0.7924 - precision: 0.8552 - recall: 0.9998
Epoch 2/15
219/219 [==============================] - 54s 245ms/step - loss: 0.5933 - accuracy: 0.7925 - precision: 0.8552 - recall: 1.0000
Epoch 3/15
219/219 [==============================] - 54s 248ms/step - loss: 0.5957 - accuracy: 0.7926 - precision: 0.8553 - recall: 1.0000
Epoch 4/15
219/219 [==============================] - 53s 244ms/step - loss: 0.5924 - accuracy: 0.7924 - precision: 0.8552 - recall: 0.9998
Epoch 5/15
219/219 [==============================] - 53s 244ms/step - loss: 0.5913 - accuracy: 0.7924 - precision: 0.8552 - recall: 0.9998
Epoch 6/15
219/219 [==============================] - 53s 244ms/step - loss: 0.5908 - accuracy: 0.7925 - precision: 0.8552 - recall: 1.0000
Epoch 7/15
219/219 [==============================] - 54s 246ms/step - loss: 0.5898 - accuracy: 0.7925 - precision: 0.8552 - recall: 1.0000
Epoch 8/15
219/219 [

In [71]:
model.evaluate(Test_X,stance_Test_Y)

32/32 [==============================] - 9s 240ms/step - loss: 0.5758 - accuracy: 0.8040 - precision: 0.8740 - recall: 1.0000


[0.5758407115936279, 0.8040000200271606, 0.8740000128746033, 1.0]

In [74]:
# f1 score for the model
# predict the stance for the test data
# pred = model.predict(Test_X)
print(stance_Train_Y)


0       1
1       1
2       1
3       1
4       0
       ..
6983    1
6984    0
6985    1
6986    1
6987    1
Name: stance, Length: 6988, dtype: int64
